# In-Memory Search Engine

Hoje vamos implementar um pequeno motor de busca para exercitar os principais conceitos vistos na disciplina até o momento. Na próxima aula iremos discutir/avaliar/comparar as soluções apresentadas pelos grupos.

**INSTRUÇÕES**

Esta atividade deve ser feita em grupos de 2 ou 3 alunos. Apenas 1 aluno precisa enviar o link do google colab pelo formulário. No formulário há campos para informar os nomes dos demais membros do grupo.

Cuidado: verifique se a opção de compartilhamento está no formato "todos com o link podem acessar". Caso não esteja compartilhado corretamente, a submissão será ignorada. ATENÇÃO: a última edição do arquivo deve ser antes do prazo de entrega.

**DESCRIÇÃO**

O motor de busca deverá ser capaz de ler os dados de um arquivo CSV com 2 colunas:

* id: o ID do documento (número inteiro)
* texto: o texto do documento

Como a intenção é fazer um motor de busca bem simples, o índice é mantido 100% em memória, na estrutura que julgarem mais adequada. Também não iremos nos preocupar com compressão de índice para economizar memória.

Os textos dos documentos devem passar por um pré-processamento antes de serem indexados.

A estratégia rankeamento deve ser baseada em TF e IDF.

O buscador não precisa fazer busca tolerante.

Há uma classe python pré-definida que deve existir, onde você deve criar os métodos. Porém, se quiser criar outras classes para serem usadas pela classe principal, não há problema.











# CSV utilizado para testes:

id,texto

1,"These are short, famous texts in English from classic sources like the Bible or Shakespeare. Some texts have word definitions and explanations to help you. Some of these texts are written in an old style of English. Try to understand them, because the English that we speak today is based on what our great, great, great, great grandparents spoke before! Of course, not all these texts were originally written in English. The Bible, for example, is a translation. But they are all well known in English today, and many of them express beautiful thoughts."

2,"I live in a house near the mountains. I have two brothers and one sister, and I was born last. My father teaches mathematics, and my mother is a nurse at a big hospital. My brothers are very smart and work hard in school. My sister is a nervous girl, but she is very kind. My grandmother also lives with us. She came from Italy when I was two years old. She has grown old, but she is still very strong. She cooks the best food! My family is very important to me. We do lots of things together. My brothers and I like to go on long walks in the mountains. My sister likes to cook with my grandmother. On the weekends we all play board games together fun. We laugh and always have a good time. I love my family very much."

3,"Jack was hungry. He walked to the kitchen. He got out some eggs. He took out some oil. He placed a skillet on the stove. Next, he turned on the heat. He poured the oil into the skillet. He cracked the eggs into a bowl. He stirred the eggs. Then, he poured them into the hot skillet. He waited while the eggs cooked. They cooked for two minutes. He heard them cooking. They popped in the oil. Next, Jack put the eggs on a plate. He placed the plate on the dining room table. Jack loved looking at his eggs. They looked pretty on the white plate. He sat down in the large wooden chair. He thought about the day ahead. He ate the eggs with a spoon. They were good. He washed the plate with dishwashing soap. Then, he washed the pan. He got a sponge damp. Finally, he wiped down the table. Next, Jack watched TV fun."

4,"Every year we go to Florida. We like to go to the beach. My favorite beach is called Emerson Beach. It is very long, with soft sand and palm trees. It is very beautiful. I like to make sandcastles and watch the sailboats go by. Sometimes there are dolphins and whales in the water! Every morning we look for shells in the sand. I found fifteen big shells last year. I put them in a special place in my room. This year I want to learn to surf. It is hard to surf, but so much fun fun fun! My sister is a good surfer. She says that she can teach me. I hope I can do it!"

5,"Lucas goes to school every day of the week. He has many subjects to go to each school day: English, art, science, mathematics, gym, and history. His mother packs a big backpack full of books and lunch for Lucas. His first class is English, and he likes that teacher very much. His English teacher says that he is a good pupil, which Lucas knows means that she thinks he is a good student. His next class is art. He draws on paper with crayons and pencils and sometimes uses a ruler. Lucas likes art. It is his favorite class. His third class is science. This class is very hard for Lucas to figure out, but he gets to work with his classmates a lot, which he likes to do. His friend, Kyle, works with Lucas in science class, and they have fun fun."

# Implementação

### Importando as Bibliotecas

In [33]:
import pandas as pd
import itertools
import functools
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Criando Classe em Memória com os principais métodos de pré-processamento, busca e rankeamento:

In [34]:
class InMemSearchEngine():
	# Setando nltk.corpus para gramática inglesa (stopwords)
	def __init__(self, path_corpus_file):
		self.path_to_corpus_file = path_corpus_file
		self.raw_corpus = pd.read_csv(self.path_to_corpus_file, index_col=0, squeeze=True).to_dict()
		self.corpus = dict()
		self.english_stop_words = set(stopwords.words("english"))
		self.init_corpus()

	# Indicação de posição sobre cada palavra
	def init_corpus(self):
		for id, texto in self.raw_corpus.items():
			inverted_doc = self.inverted_index(texto)
			self.inverted_index_add(self.corpus, id, inverted_doc)

	# Separação dos textos por palavras
	def word_split(self, text):
		word_list = []
		wcurrent = []
		windex = None
		for i, c in enumerate(text):
			if c.isalnum():
				wcurrent.append(c)
				windex = i
			elif wcurrent:
				word = u''.join(wcurrent)
				word_list.append((windex - len(word) + 1, word))
				wcurrent = []
		if wcurrent:
			word = u''.join(wcurrent)
			word_list.append((windex - len(word) + 1, word))
		return word_list

	# Filtragem das palavras mais relevantes dos textos, a partir dos stopwords
	def words_cleanup(self, words):
		cleaned_words = []
		for index, word in words:
			if len(word) < 1 or word in self.english_stop_words:
				continue
			cleaned_words.append((index, word))
		return cleaned_words

	# Normalização das palavras, para facilitar buscas
	def words_normalize(self, words):
		normalized_words = []
		for index, word in words:
			wnormalized = word.lower()
			normalized_words.append((index, wnormalized))
		return normalized_words
	
	# Aplicação de Stemmer, separação, normalização e filtragem de palavras nos textos
	def word_index(self, text):
		ps = PorterStemmer()
		words = self.word_split(text)
		words = [(pos, ps.stem(word)) for pos, word in words]
		words = self.words_normalize(words)
		words = self.words_cleanup(words)
		return words

	# Cria um indice invertido para as palavras de um documento
	def inverted_index(self, text):
		inverted = {}
		for index, word in self.word_index(text):
			locations = inverted.setdefault(word, [])
			locations.append(index)
		return inverted

	# Mescla o indice invertido de varios documentos
	def inverted_index_add(self, inverted, doc_id, doc_index):
		for word, locations in doc_index.items():
			indices = inverted.setdefault(word, {})
			indices[doc_id] = locations
		return inverted

	# Ordenação e listagem de Rankeamento
	def rank(self, words, docs):
		if (len(docs) <= 1):
			return list(docs)

		def rank_sort(words):	
			def rank_sort_func(doc1, doc2):	
				doc1_aux = 0
				doc2_aux = 0
				for word in words:
					if (doc1 not in self.corpus[word]):
						if doc2 in self.corpus[word]:
							doc2_aux += 1
					elif (doc2 not in self.corpus[word]):
						if doc1 in self.corpus[word]:
							doc1_aux += 1
					elif (len(self.corpus[word][doc1]) > len(self.corpus[word][doc2])):
						doc1_aux += 1
					else:
						doc2_aux += 1
				return doc2_aux - doc1_aux
			return rank_sort_func
		
		ranked = sorted(docs, key=functools.cmp_to_key(rank_sort(words)))
		return ranked

	def search(self, query, op="AND"):
		words = [word for _, word in self.word_index(query) if word in self.corpus]
		results = [set(self.corpus[word].keys()) for word in words]
		if (op == "AND"):
			results = functools.reduce(lambda x, y: x & y, results) if results else []
		else:
			results = [list(docs) for docs in results]
			results = set(itertools.chain.from_iterable(results))
		ranked_results = self.rank(words, results)
		print("unranked_results:",results)
		return ranked_results


### Main para testes de Buscas e Rankeamento:

In [35]:
# O rankeamento preza mais por Revocação do que a Precisão.
def main():
  path_corpus_file = './corpus.csv'
  se = InMemSearchEngine(path_corpus_file)

  query = 'food'
  print("query:", query)
  results = se.search(query)
  print("ranked_results:", results)

  print()

  query = 'cooks food'
  print("query:", query)
  results = se.search(query)
  print("ranked_results:", results)

  print()

  query = 'cooks food'
  print("query:", query)
  results = se.search(query, op="OR")
  print("ranked_results:", results)

  print()

  query = 'fun'
  print("query:", query)
  results = se.search(query)
  print("ranked_results:", results)

main()

query: food
unranked_results: {2}
ranked_results: [2]

query: cooks food
unranked_results: {2}
ranked_results: [2]

query: cooks food
unranked_results: {2, 3}
ranked_results: [2, 3]

query: fun
unranked_results: {2, 3, 4, 5}
ranked_results: [4, 5, 2, 3]


### Teste de ordem e/ou proximidade entre as palavras (em desenvolvimento):